In [1]:
trueCases = {}
with open("GrandTruth.csv") as GrandTruth:
    for line in GrandTruth:
        lineListed = line.split()
        try:
            trueCases[lineListed[1]].append(lineListed[2])
        except KeyError:
            trueCases[lineListed[1]] = [lineListed[2]]

In [2]:
import csv 
findings = []   
with open("Results(3.0per10000).csv") as csv_file:
        Mode = ""
        Reference_Dictionary = {}
        Temporary_Dictionary = {}
        Temporary_Site_List = []
        Reader = csv.reader(csv_file, delimiter = ";")
        for Row in Reader:
            if Row == []:
                continue;
            Mode = ("Read Sequence" if Row[0] == "Cell ID" else 
                     ("Read Factor" if Row[0] == "HIT ID" else Mode))
            if (Row[0] == "Enzyme ID") or (Row[0] == "HIT ID"):
                continue;
            elif Row[0] == "NULL":
                Temporary_Dictionary["Sites"] = Temporary_Site_List
                Reference_Dictionary[Temporary_Dictionary["SubUnit"]] = {Key : Temporary_Dictionary[Key] for Key in Temporary_Dictionary.keys()}
                Temporary_Dictionary = {}
                Temporary_Site_List = []
                continue;
            if Mode == "Read Sequence":
                Temporary_Dictionary = {"Cell ID": Row[0], "Pathway": Row[1], "Reaction": Row[2],
                                        "Enzyme": Row[3], "SubUnit": Row[4], "Promoter": Row[5], "Hash": Row[7]}
            elif Mode == "Read Factor":
                Temporary_Site_List.append([Row[0], Row[1], Row[3]])
        findings = Reference_Dictionary

In [3]:
truePositives = {}
falseNegatives = {}
for promoter in trueCases.keys():
    truePositives[promoter] = {}
    falseNegatives[promoter] = []
    for regulator in trueCases[promoter]:
        found = False
        for hit in findings[promoter]["Sites"]:
            if hit[0] == regulator[:-1].upper():
                found = True
                try:
                    truePositives[promoter][regulator].append(hit)
                except KeyError:
                    truePositives[promoter][regulator] = [hit]
        if found == False:
            falseNegatives[promoter].append(regulator)

In [4]:
falsePositiveSites = {}
falsePositiveFactors = {}
for promoter in findings.keys():
    totalSites = len(findings[promoter]["Sites"])
    totalFactors = len({factor[0] for factor in findings[promoter]["Sites"]})
    positiveSites = 0
    positiveFactors = len(truePositives[promoter])
    for regulator in truePositives[promoter].values():
        positiveSites += len(regulator)
    falsePositiveSites[promoter] = totalSites - positiveSites
    falsePositiveFactors[promoter] = totalFactors - positiveFactors

In [5]:
print("Promoter | True Positive | False Negative | False Positive Factors | False Positive Sites")
totalTrue = 0
totalFalseNeg = 0
totalFalsePos = 0
for promoter in trueCases.keys():
    tPos = len(truePositives[promoter])
    totalTrue += tPos
    fNeg = len(falseNegatives[promoter])
    totalFalseNeg += fNeg
    totalFalsePos += falsePositiveFactors[promoter]
    print("{}\t{}\t{}\t{}\t{}\n".format(promoter, tPos, fNeg, falsePositiveFactors[promoter], falsePositiveSites[promoter]))
print("True Positive to False Negative Factor ratio is {}".format(totalTrue/totalFalseNeg))
print("True Positive to False Positive Factor ratio is {}".format(totalTrue/totalFalsePos))

Promoter | True Positive | False Negative | False Positive Factors | False Positive Sites
HXK2	9	9	71	122

PGI1	3	10	64	109

PFK1	6	4	43	63

PFK2	8	6	76	133

FBA1	8	9	56	107

TPI1	5	9	62	84

TDH2	11	6	69	165

TDH3	12	12	48	80

PGK1	8	17	58	92

GPM1	4	10	64	144

ENO2	5	12	48	70

CDC19	18	9	71	152

PCK1	10	10	79	150

ENO1	14	10	95	222

FBP1	6	8	79	148

ZWF1	18	20	81	258

SOL3	2	4	69	125

SOL4	6	18	69	107

GND1	7	8	83	150

RPE1	4	2	46	71

RKI1	14	5	88	181

TKL1	9	10	77	131

TAL1	7	6	65	130

PDC1	13	10	70	113

ADH1	10	11	70	143

PDA1	3	4	80	141

PDB1	0	2	84	148

LAT1	5	3	80	120

LPD1	4	3	60	118

PDX1	1	4	80	126

CIT1	9	14	94	242

ACO1	6	10	89	192

IDH1	12	9	77	142

IDH2	4	4	76	142

KGD1	3	3	81	148

KGD2	2	4	75	115

LSC1	0	1	78	118

LSC2	4	4	73	115

SDH1	5	7	93	190

SDH2	2	0	75	121

SDH3	0	1	67	109

SDH4	1	1	63	89

FUM1	0	2	85	160

MDH1	7	3	96	266

CIT2	9	7	78	150

ICL1	5	10	76	130

MLS1	10	10	60	100

MDH2	12	15	68	153

ADH2	4	10	72	135

ALD4	9	4	73	215

ALD6	13	14	80	244

ACS1	11	11	75	180

In [6]:
# tfSet = set()
# with open("Yeastract_TFList.txt") as txtFile:
#     for line in txtFile:
#         lineListed = line.split()
#         tfSet.add(lineListed[0])
# 
# with open("TF_Dictionary.txt", "w") as dictionary:
#     for item in tfSet:
#         dictionary.write("{}\t{}\n".format(item,item[:-1].upper()))
# 